Thanks to [LibAUC](https://github.com/Optimization-AI/LibAUC)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/Grad/CheXpert-v1.0-small.zip > /dev/null

In [ ]:
# !pip install vit-pytorch

In [3]:
!pip install libauc

In [4]:
pip install timm

     |████████████████████████████████| 431 kB 29.3 MB/s 


In [5]:
import time
import IPython

import cv2
import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam, lr_scheduler

from torchsummary import summary

from sklearn.metrics import roc_auc_score

cuda = torch.device('cuda:0')

In [6]:
!mv /content/clean_4.csv /content/CheXpert-v1.0-small

In [12]:
save_weights_path = '/content/drive/MyDrive/Grad/aucm_multi_label_pretrained_model_18.pth'
load_weights_path = '/content/drive/MyDrive/Grad/aucm_multi_label_pretrained_model_17.pth'

# Original LibAUC

In [13]:
from libauc.losses import AUCM_MultiLabel
from libauc.optimizers import PESG
from chexpert import CheXpert

In [14]:
import timm

In [15]:
avail_pretrained_models = timm.list_models(pretrained=True)
avail_pretrained_models


['adv_inception_v3',
 'bat_resnext26ts',
 'beit_base_patch16_224',
 'beit_base_patch16_224_in22k',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_224_in22k',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'botnet26t_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'convit_base',
 'convit_small',
 'convit_tiny',
 'convmixer_768_32',
 'convmixer_1024_20_ks9_p14',
 'convmixer_1536_20',
 'convnext_base',
 'convnext_base_384_in22ft1k',
 'convnext_base_in22ft1k',
 'convnext_base_in22k',
 'convnext_large',
 'convnext_large_384_in22ft1k',
 'convnext_large_in22ft1k',
 'convnext_large_in22k',
 'convnext_small',
 'convnext_tiny',
 'convnext_xlarge_384_in22ft1k',
 'convnext_xlarge_in22ft1k',
 'convnext_xlarge_in22k',
 'crossvit_9_240',
 'crossv

In [ ]:
# dataloader
root = '/content/CheXpert-v1.0-small/'
# Index: -1 denotes multi-label mode including 5 diseases
traindSet = CheXpert(csv_path=root+'clean_4.csv', image_root_path=root, use_upsampling=False, use_frontal=False, image_size=224, mode='train', class_index=-1, verbose=False)
testSet =  CheXpert(csv_path=root+'valid.csv',  image_root_path=root, use_upsampling=False, use_frontal=False, image_size=224, mode='valid', class_index=-1, verbose=False)
trainloader =  torch.utils.data.DataLoader(traindSet, batch_size=32, num_workers=2, shuffle=True)
testloader =  torch.utils.data.DataLoader(testSet, batch_size=32, num_workers=2, shuffle=False)

# paramaters
BATCH_SIZE = 32


lr = 0.002# using smaller learning rate is better
gamma = 1700
imratio = traindSet.imratio_list 
weight_decay = 1e-4
margin = 0.95

#model = DenseNet121(pretrained=True, last_activation=None, activations='relu', num_classes=5)
model = timm.create_model('coat_mini', num_classes=5, pretrained=True)
model = model.cuda()

# define loss & optimizer
Loss = AUCM_MultiLabel(imratio=imratio, num_classes=5)
optimizer = PESG(model, 
                 a=Loss.a, 
                 b=Loss.b, 
                 alpha=Loss.alpha, 
                 lr=lr, 
                 gamma=gamma, 
                 margin=margin, 
                 weight_decay=weight_decay, device='cuda')

model.load_state_dict(torch.load(load_weights_path))

# training
# diff_threshold = 0.01
# const_threshold = 3

val_auc_mean = 0.8806
best_val_auc = 0.8806 
const_cnt = 0
for epoch in range(1):     
    for idx, data in enumerate(trainloader):
      train_data, train_labels = data
      train_data, train_labels  = train_data.cuda(), train_labels.cuda()
      y_pred = model(train_data)
      y_pred = torch.sigmoid(y_pred)
      loss = Loss(y_pred, train_labels)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
        
      # validation  
      if idx % 400 == 0:
         model.eval()
         with torch.no_grad():    
              test_pred = []
              test_true = [] 
              for jdx, data in enumerate(testloader):
                  test_data, test_labels = data
                  test_data = test_data.cuda()
                  y_pred = model(test_data)
                  y_pred = torch.sigmoid(y_pred)
                  test_pred.append(y_pred.cpu().detach().numpy())
                  test_true.append(test_labels.numpy())
            
              test_true = np.concatenate(test_true)
              test_pred = np.concatenate(test_pred)
              val_auc_mean =  roc_auc_score(test_true, test_pred) 
              model.train()

              if best_val_auc < val_auc_mean:
                  #const_cnt = 0
                  best_val_auc = val_auc_mean
                  torch.save(model.state_dict(), save_weights_path)
             # else:
                  #const_cnt += 1            

              # val_diff = val_auc_mean-best_val_auc
              # if ((val_diff < 0) & (abs(val_diff) > diff_threshold)) or (const_cnt >= const_threshold):
              #     optimizer.update_regularizer(decay_factor=10)
              #     const_cnt = 0 

              print ('Epoch=%s, BatchID=%s, Val_AUC=%.4f, Best_Val_AUC=%.4f'%(epoch, idx, val_auc_mean, best_val_auc))

Epoch=0, BatchID=0, Val_AUC=0.8851, Best_Val_AUC=0.8851
Epoch=0, BatchID=400, Val_AUC=0.8895, Best_Val_AUC=0.8895
Epoch=0, BatchID=800, Val_AUC=0.8878, Best_Val_AUC=0.8895
Epoch=0, BatchID=1200, Val_AUC=0.8809, Best_Val_AUC=0.8895
Epoch=0, BatchID=1600, Val_AUC=0.8839, Best_Val_AUC=0.8895
Epoch=0, BatchID=2000, Val_AUC=0.8845, Best_Val_AUC=0.8895
Epoch=0, BatchID=2400, Val_AUC=0.8841, Best_Val_AUC=0.8895
Epoch=0, BatchID=2800, Val_AUC=0.8866, Best_Val_AUC=0.8895
Epoch=0, BatchID=3200, Val_AUC=0.8849, Best_Val_AUC=0.8895
Epoch=0, BatchID=3600, Val_AUC=0.8857, Best_Val_AUC=0.8895
Epoch=0, BatchID=4000, Val_AUC=0.8835, Best_Val_AUC=0.8895
Epoch=0, BatchID=4400, Val_AUC=0.8859, Best_Val_AUC=0.8895


In [ ]:
model.eval()
with torch.no_grad():    
    test_pred = []
    test_true = [] 
    for jdx, data in enumerate(testloader):
        test_data, test_labels = data
        test_data = test_data.cuda()
        y_pred = model(test_data)
        y_pred = torch.sigmoid(y_pred)
        test_pred.append(y_pred.cpu().detach().numpy())
        test_true.append(test_labels.numpy())
  
    test_true = np.concatenate(test_true)
    test_pred = np.concatenate(test_pred)

In [ ]:
labels = ['Cardiomegaly', 'Edema', 'Consolidation', 'Atelectasis', 'Pleural Effusion']

results = pd.DataFrame(index=labels)


scores = []
for i in range(5):
    score = roc_auc_score(test_true[:, i], test_pred[:, i])
    scores.append(score)
    
results['AUC'] = scores

In [ ]:
results

,AUC
Cardiomegaly,0.859626
Edema,0.895833
Consolidation,0.907537
Atelectasis,0.823832
Pleural Effusion,0.888134


In [ ]:
results['AUC'].mean()

0.8749923690913606

In [ ]:
results

,AUC
Cardiomegaly,0.859848
Edema,0.894643
Consolidation,0.920404
Atelectasis,0.837690
Pleural Effusion,0.894361


In [ ]:
results['AUC'].mean()


0.8813894911026875